In [1]:
import ipdb
import os
import re
import random
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import razdel
from razdel import tokenize
from collections import Counter, namedtuple
from enum import IntEnum

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler
from torchmetrics.classification.accuracy import Accuracy

from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping  
from transformers import XLMRobertaTokenizer, XLMRobertaModel

import xml.etree.ElementTree as ET
from ipymarkup import show_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN

In [2]:
df = pd.read_csv('../irregular_train_fixed.csv')
df2 = pd.read_csv('../regular_train_fixed.csv')
df = pd.concat([df, df2], axis=0)

In [3]:
df_test = pd.read_csv('../irregular_test_fixed.csv')
df2_test = pd.read_csv('../regular_test_fixed.csv')
df_test = pd.concat([df_test, df2_test], axis=0)

In [4]:
df

,Unnamed: 0,text,target,tar_pos_l,tar_pos_r,amount,currency,amount_pos_l,amount_pos_r,currency_pos_l,currency_pos_r,filename
0,0,DUAL SERVICE MEMORANDUM OF AGREEMENT FOR EMPLO...,"['$ 1000', '$ 1332.3', '$ 3150.75']","[1386, 1610, 1110]","[1395, 1619, 1119]","['1000', '1332.3', '3150.75']","['$', '$', '$']","[1387, 1611, 1111]","[1395, 1619, 1119]","[1386, 1610, 1110]","[1387, 1611, 1111]",data/rdf_unregular_eng/train\100031_ForML2.pdf...
1,1,DUAL SERVICE MEMORANDUM OF AGREEMENT FOR EMPLO...,['$ 2108.45'],[1098],[1107],['2108.45'],['$'],[1099],[1107],[1098],[1099],data/rdf_unregular_eng/train\100068_ForML1.pdf...
2,2,07-0088 DUAL SERVICE MEMORANDUM OF AGREEMENT F...,"['$ 2708.3', '$ 1294.15', '$ 1294.15', '$ 1414...","[1722, 1851, 1018, 1168, 1190]","[1731, 1860, 1027, 1177, 1199]","['2708.3', '1294.15', '1294.15', '1414.15', '2...","['$', '$', '$', '$', '$']","[1723, 1852, 1019, 1169, 1191]","[1731, 1860, 1027, 1177, 1199]","[1722, 1851, 1018, 1168, 1190]","[1723, 1852, 1019, 1169, 1191]",data/rdf_unregular_eng/train\100086_ForML1.pdf...
3,3,DUAL SERVICE MEMORANDUM OF AGREEMENT FOR EMPLO...,['$ 3889.5'],[1967],[1976],['3889.5'],['$'],[1968],[1976],[1967],[1968],data/rdf_unregular_eng/train\100120_ForML1.pdf...
4,4,DUAL SERVICE MEMORANDUM OF AGREEMENT FOR EMPLO...,['$ 1296.5'],[1731],[1740],['1296.5'],['$'],[1732],[1740],[1731],[1732],data/rdf_unregular_eng/train\100121_ForML1.pdf...
...,...,...,...,...,...,...,...,...,...,...,...,...
4528,4528,"Windows (4.5%, 11/243), Nokia (2.5%, 6/243), a...","['$ 0.99', '$ 15.99', '$ 4.99', 'CAN $ 3.15']","[578, 587, 662, 552]","[583, 593, 667, 561]","['0.99', '15.99', '4.99', '3.15']","['$', '$', '$', 'CAN $']","[579, 588, 663, 557]","[583, 593, 667, 561]","[578, 587, 662, 552]","[579, 588, 663, 557]",data/rdf_regular_eng/Train\CAN_dollar (35).doc...
4529,4529,"In 2006, Rights & Democracy concluded 3-year p...",['CAN$ 200000'],[89],[101],['200000'],['CAN$'],[94],[101],[89],[93],data/rdf_regular_eng/Train\CAN_dollar (36).doc...
4530,4530,Mentioning that the Stolt LNGaz project was va...,"['US$ 600000000', 'CAN$ 800000000']","[64, 94]","[78, 109]","['600000000', '800000000']","['US$', 'CAN$']","[67, 98]","[78, 109]","[64, 94]","[67, 98]",data/rdf_regular_eng/Train\CAN_dollar (37).doc...
4531,4531,"CAN$1,000,000 is approximately equal to $709,3...","['$ 709319', 'US$ 70932', 'CAN$ 1000000', 'CAN...","[40, 91, 0, 53, 132, 138]","[48, 100, 13, 64, 137, 149]","['709319', '70932', '1000000', '100000', '1', ...","['$', 'US$', 'CAN$', 'CAN$', 'CAN$', 'US$']","[41, 94, 4, 57, 136, 141]","[48, 100, 13, 64, 137, 149]","[40, 91, 0, 53, 132, 138]","[41, 94, 4, 57, 136, 141]",data/rdf_regular_eng/Train\CAN_dollar (4).docx...


In [5]:
df_test

,Unnamed: 0,text,target,tar_pos_l,tar_pos_r,amount,currency,amount_pos_l,amount_pos_r,currency_pos_l,currency_pos_r,filename
0,0,DUAL SERVICE MEMORANDUM OF AGREEMENT FOR EMPLO...,['$ 1000'],[1505],[1515],['1000'],['$'],[1507],[1515],[1505],[1506],data/rdf_unregular_eng/test\100413_ForML2.pdf.xml
1,1,"AGREEMENT THIS AGREEMENT, macle and entered in...",['S 3700000'],[374],[384],['3700000'],['S'],[375],[384],[374],[375],data/rdf_unregular_eng/test\100533_ForML1.pdf.xml
2,2,This Instrument Prepared By: Middle Tennessee ...,['$_____ 27600'],[1947],[1959],['27600'],['$_____'],[1953],[1959],[1947],[1953],data/rdf_unregular_eng/test\100619_ForML1.pdf.xml
3,3,DUAL SERVICE MEMORANDUM OF AGREEMENT FOR EMPLO...,['$ 2014.75'],[1464],[1473],['2014.75'],['$'],[1465],[1473],[1464],[1465],data/rdf_unregular_eng/test\100716_ForML1.pdf.xml
4,4,DUAL SERVICE MEMORANDUM OF AGREEMENT FOR EMPLO...,"['$ 110', '$ 110']","[1808, 968]","[1816, 976]","['110', '110']","['$', '$']","[1809, 969]","[1816, 976]","[1808, 968]","[1809, 969]",data/rdf_unregular_eng/test\100732_ForML1.pdf.xml
...,...,...,...,...,...,...,...,...,...,...,...,...
1593,1593,As part of his compensation for the financial ...,"['CAN $ 1400000', 'CAN $ 2285000', 'CAN $ 3675...","[197, 213, 232, 342, 393, 447]","[211, 227, 246, 351, 402, 456]","['1400000', '2285000', '3675000', '3.50', '4.6...","['CAN $', 'CAN $', 'CAN $', 'CAN $', 'CAN $', ...","[202, 218, 237, 347, 398, 452]","[211, 227, 246, 351, 402, 456]","[197, 213, 232, 342, 393, 447]","[202, 218, 237, 347, 398, 452]",data/rdf_regular_eng/Test\CAN_dollar (38).docx...
1594,1594,"In that case, the plaintiff purchased a house ...","['CAN$ 88000', 'CAN$ 271000', 'CAN$ 121000', '...","[50, 254, 293, 333, 403]","[60, 265, 304, 344, 413]","['88000', '271000', '121000', '150000', '18900']","['CAN$', 'CAN$', 'CAN$', 'CAN$', 'CAN$']","[54, 258, 297, 337, 407]","[60, 265, 304, 344, 413]","[50, 254, 293, 333, 403]","[54, 258, 297, 337, 407]",data/rdf_regular_eng/Test\CAN_dollar (5).docx.xml
1595,1595,If you have been outside Canada for: 24 hours ...,"['CAN$ 200', 'CAN$ 800']","[72, 333]","[79, 340]","['200', '800']","['CAN$', 'CAN$']","[76, 337]","[79, 340]","[72, 333]","[76, 337]",data/rdf_regular_eng/Test\CAN_dollar (7).docx.xml
1596,1596,Some operations of Bombardier are conducted un...,['CAN$ 20000000'],[1198],[1212],['20000000'],['CAN$'],[1202],[1212],[1198],[1202],data/rdf_regular_eng/Test\CAN_dollar (8).docx.xml


In [6]:
def prepare_data(df):
    data = list()
    tar_pos_l = df['tar_pos_l'].values
    tar_pos_r = df['tar_pos_r'].values
    text = df['text'].values
    for i in range(df.shape[0]):
        text_i = text[i]
        try:
            tar_pos_l_i = list(map(int, tar_pos_l[i].strip("'[]'").split(', ')))
            tar_pos_r_i = list(map(int, tar_pos_r[i].strip("'[]'").split(', ')))
        except ValueError:
            continue
        data.append( (text_i, [(l, r, 'TAR') for l, r in zip(tar_pos_l_i, tar_pos_r_i)]) )
    return data

In [7]:
data = prepare_data(df)
data_test = prepare_data(df_test)

In [8]:
show_box_markup(data[0][0], data[0][1], palette=palette(TAR=BLUE, ORG=RED, LOC=GREEN))

In [9]:
class Label(IntEnum):
    OUTER = 0
    BEGIN = 1
    INNER = 2

Sample = namedtuple("Sample", "text,tokens,spans,labels")
it = 0
def text_span_to_sample(text, spans):
    labels = []
    tokens = list(tokenize(text))
    
    for token in tokens:
        label = Label.OUTER
        for span in spans:
            # Начало или часть какого-то спана
            # Не совсем корректно из-за возможных различий разметки и токенизации
            span_begin, span_end, tag = span
            if token.start == span_begin:
                label = Label.BEGIN
            elif token.start > span_begin and token.stop <= span_end:
                label = Label.INNER
        labels.append(label)

    # Проверяем инвариант отсутствия последовательных пар (0, 2)
    try:
        assert len([1 for i in range(len(labels) - 1) if (labels[i], labels[i+1]) == (0, 2)]) == 0
        return Sample(text, tokens, spans, labels)
    except AssertionError:
        pass
    

samples = []
for text, spans in data:
    samples.append(text_span_to_sample(text, spans))

test_samples = []
for text, spans in data_test:
    test_samples.append(text_span_to_sample(text, spans))
    
show_box_markup(samples[2].text, samples[2].spans, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))
print(samples[2].labels)
print(len(samples))
print(len(test_samples))

[<Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OU

In [10]:
random.shuffle(test_samples)
val = test_samples[:1000]
test = test_samples[1000:]
train = samples

In [11]:
def remove_none(sample):
    for entry in sample:
        if entry == None:
            sample.remove(entry)

for _ in range(3):
    remove_none(val)
    remove_none(test)
    remove_none(train)

In [12]:
char_set = ["<pad>", "<unk>"] + list({ch for sample in train for token in sample.tokens for ch in token.text})
print(char_set)

['<pad>', '<unk>', '灣', '尔', 'ı', 'ت', 'ğ', 'Û', '台', 'Θ', 'ো', '�', 'Á', 'ث', '奧', '±', '国', 'B', 'շ', 'ສ', 'ờ', 'ո', '§', 'ř', 'ǒ', 'স', 'à', 'Ե', '飛', 'V', 'א', 'і', 'р', '极', '%', '৳', '球', 'ო', 'Å', 'ា', 'х', 'ৎ', 'ư', 'Π', '鏡', 'ی', '…', 'ি', '發', '“', 'ڑ', '娱', '\ufeff', 'ו', '⅛', 'Č', '動', ',', 'Ф', 'ک', 'ד', 'Н', '牌', 'è', 'С', '网', '&', 'Ø', '篙', '少', '恩', '।', '기', '庭', '文', 'ი', 'ổ', 'শ', '燃', '银', 'ą', 'ε', 'イ', '’', 'Í', 'd', '￡', 'ด', '技', 'á', 'ă', 'ス', 'ū', 'А', 'ō', 'ុ', 'ȉ', '\xad', 'й', '$', 'ئ', 'Հ', 'հ', 'ب', 'س', '御', '司', '交', '!', 'Λ', 'য', 'X', '≤', 'ვ', 'Î', '–', '典', 'Б', '匚', 'บ', 'ไ', 'G', 't', 'ù', '®', '\\', 'ر', 'ã', 'ố', '■', 'ž', '`', '京', 'z', 'ī', '格', 'パ', '制', 'ť', 'є', 'م', 'ọ', '東', 'տ', 'ঢ', 'の', 'ồ', '氣', '□', 'ᥢ', '₡', '№', ')', '保', '₵', 'ে', 'ắ', '厂', '´', 'ů', '¥', "'", 'び', '尼', '￦', '|', 'ч', 'å', 'ラ', 'מ', 'v', 'श', 'ฮ', 'ל', 'Ľ', 'ц', '橘', 'を', 'В', 'َ', 'Ú', 'é', 'ί', '浙', '行', 'F', 'i', 'ল', 'น', 'ິ', '乐', 'থ', '₾', 'এ', 'ッ', 'פ', '城

In [15]:
sample = samples[0]

max_seq_len = 0
max_char_seq_len = 0

for sample in samples:
    max_seq_len = max(len(sample.tokens), max_seq_len)
    for token in sample.tokens:
        max_char_seq_len = max(len(token.text), max_char_seq_len)
max_seq_len

11751

In [16]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler

class NerCharsDataset(Dataset):
    def __init__(self, samples, char_set, max_seq_len=500, max_char_seq_len=50):
        assert len(samples) != 0
        self.samples = []
        self.tokens = []
        self.texts = []
        for sample in samples:
            inputs = torch.zeros((max_seq_len, max_char_seq_len), dtype=torch.long)
            for token_num, token in enumerate(sample.tokens[:max_seq_len]):
                for ch_num, ch in enumerate(token.text[:max_char_seq_len]):
                    char_index = char_set.index(ch) if ch in char_set else char_set.index("<unk>")
                    inputs[token_num][ch_num] = char_index
            labels = torch.zeros((max_seq_len), dtype=torch.long)
            input_labels = [int(i) for i in sample.labels[:max_seq_len]]
            labels[:len(input_labels)] = torch.LongTensor(input_labels)
            self.samples.append((torch.LongTensor(inputs), torch.LongTensor(labels)))
            self.tokens.append(sample.tokens[:max_seq_len])
            self.texts.append(sample.text)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        return self.samples[index]

    def get_tokens(self, index):
        return self.tokens[index]
    
    def get_text(self, index):
        return self.texts[index]


BATCH_SIZE = 32

train_data = NerCharsDataset(train, char_set)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

val_data = NerCharsDataset(val, char_set)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = NerCharsDataset(test, char_set)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [17]:
batch = next(iter(train_loader))

In [18]:
inputs, labels = batch
print(inputs.size())
print(labels.size())

# inputs: batch_size x num_words x num_chars
# labels: batch_size x num_words

torch.Size([32, 500, 50])
torch.Size([32, 500])


In [19]:
inputs[0]

tensor([[400,   0,   0,  ...,   0,   0,   0],
        [347,  60, 318,  ...,   0,   0,   0],
        [  5, 154, 286,  ...,   0,   0,   0],
        ...,
        [  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   0,   0,   0]])

In [20]:
import torch
import torch.nn as nn
import torchmetrics
from pytorch_lightning import LightningModule
from torchmetrics import Accuracy
 

class TemplateModel(LightningModule):
    def __init__(self):
        super().__init__()
        
        self.loss = nn.CrossEntropyLoss()
        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
    
    def forward(self, inputs, labels):
        raise NotImplementedError("forward not implemented")
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return [optimizer]
    
    def training_step(self, batch, _):
        inputs, labels = batch
        loss, logits = self(inputs, labels)
        return loss
    
    def validation_step(self, batch, _):
        inputs, labels = batch
        val_loss, logits = self(inputs, labels)
        self.valid_accuracy.update(logits, labels)
        self.log("val_loss", val_loss, prog_bar=True)
        self.log("val_acc", self.valid_accuracy)

    def validation_epoch_end(self, outs):
        self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, labels = batch
        test_loss, logits = self(inputs, labels)
        self.test_accuracy.update(logits, labels)
        self.log("test_loss", test_loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy)

    def test_epoch_end(self, outs):
        self.log("test_acc_epoch", self.test_accuracy.compute(), prog_bar=True)

In [21]:
def get_spans(labels, tokens):
    spans = []
    for i, (label, token) in enumerate(zip(labels, tokens)):
        if label == 1:
            spans.append((token.start, token.stop, "PER"))
        elif label == 2:
            assert len(spans) != 0, "Incorrect label sequence: {}".format(labels)
            old_begin, _, old_tag = spans[-1]
            spans[-1] = (old_begin, token.stop, old_tag)
    return spans


def compare_span_sets(left_spans, right_spans):
    exact, partial, missing = 0, 0, 0
    for left_span in left_spans:
        is_missing = True
        for right_span in right_spans:
            if left_span == right_span:
                exact += 1
                is_missing = False
                break
            ls, le, _ = left_span
            rs, re, _ = right_span
            # [ls le] [rs re]
            # [rs re] [ls le]
            if not (ls <= le <= rs <= re or rs <= re <= ls <= le):
                is_missing = False
                partial += 1
                break            
        if is_missing:
            missing += 1
    return exact, partial, missing


def calc_metrics(true_labels, predicted_labels, tokens):
    # Метрики классификации
    one_tp, one_fp, one_fn = 0, 0, 0
    for true, predicted in zip(true_labels, predicted_labels):
        for l1, l2 in zip(true, predicted):
            if l1 == 1 and l2 == 1:
                one_tp += 1
            elif l1 != 1 and l2 == 1:
                one_fp += 1
            elif l1 == 1 and l2 !=1:
                one_fn += 1
    if one_tp + one_fp == 0:
        print("No positives!")
    else:
        print("1 Precision: {}, 1 Recall: {}".format(float(one_tp)/(one_tp + one_fp), float(one_tp)/(one_tp + one_fn)))

    # Специализированные метрики
    e, p, m, s = 0, 0, 0, 0
    for (true, predicted), sample_tokens in zip(zip(true_labels, predicted_labels), tokens):
        true_spans = get_spans(true, sample_tokens)
        predicted_spans = get_spans(predicted, sample_tokens)
        exact, partial, missing = compare_span_sets(true_spans, predicted_spans)
        _, _, spurius = compare_span_sets(predicted_spans, true_spans)
        e += exact
        p += partial
        m += missing #fn
        s += spurius #fp
    print("Exact: {}, partial: {}, missing: {}, spurius: {}".format(e, p, m, s))
            


def predict(model, test_loader, show_sample_index=1):
    model.eval()
    all_true_labels, all_predicted_labels, all_tokens, all_texts = [], [], [], []
    for batch_index, batch in enumerate(test_loader):
        inputs, true_labels = batch
        batch_size = inputs.size(0)
        _, logits = model(inputs, true_labels)
        predicted_labels = logits.max(dim=1)[1].detach().cpu()

        # Убираем неконсистентность BIO
        for sample_num, sample in enumerate(predicted_labels):
            for token_num, label in enumerate(sample):
                if token_num == 0 and label == 2:
                    predicted_labels[sample_num][0] = 1
                    continue
                prev_label = sample[token_num - 1]
                if label == 2 and prev_label == 0:
                    predicted_labels[sample_num][token_num] = 1

        all_true_labels.extend(true_labels)
        all_predicted_labels.extend(predicted_labels)
        for i in range(batch_size):
            all_tokens.append(test_data.get_tokens(batch_index * batch_size + i))
            all_texts.append(test_data.get_text(batch_index * batch_size + i))

    calc_metrics(all_true_labels, all_predicted_labels, all_tokens)
    print("PREDICTED:")
    show_box_markup(all_texts[show_sample_index],
                    get_spans(all_predicted_labels[show_sample_index], all_tokens[show_sample_index]),
                    palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))

In [22]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [32]:
import torch
from torch import nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

class CharFFLstmModel(TemplateModel):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3,
                 word_embedding_dim=16, lstm_embedding_dim=256, char_max_seq_len=50):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(char_embedding_dim * char_max_seq_len, word_embedding_dim)
        self.relu = nn.ReLU()
        self.lstm_layer = nn.LSTM(word_embedding_dim, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)

    def forward(self, inputs, labels):
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        projections = self.relu(self.linear(projections))
        #projections = self.dropout(projections)
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        logits = self.out_layer(output)
        logits = logits.transpose(1, 2)
        loss = self.loss(logits, labels)
        return loss, logits


char_ff_lstm_model = CharFFLstmModel(len(char_set))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=8,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    enable_checkpointing=True,
    accumulate_grad_batches=1,
    max_epochs=150,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(char_ff_lstm_model, train_loader, val_loader)
trainer.test(char_ff_lstm_model, test_loader)

D:\Anaconda\envs\nlp\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params
------------------------------------------------------
0 | loss             | CrossEntropyLoss | 0     
1 | valid_accuracy   | Accuracy         | 0     
2 | test_accuracy    | Accuracy         | 0     
3 | embeddings_layer | Embedding        | 3.0 K 
4 | dropout          | Dropout 

Sanity Checking: 0it [00:00, ?it/s]

D:\Anaconda\envs\nlp\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\Anaconda\envs\nlp\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 8 records. Best score: 0.039. Signaling Trainer to stop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\Anaconda\envs\nlp\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9920492768287659
     test_acc_epoch         0.9920492768287659
        test_loss          0.022869132459163666
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.022869132459163666,
  'test_acc': 0.9920492768287659,
  'test_acc_epoch': 0.9920492768287659}]

In [33]:
predict(char_ff_lstm_model, test_loader)

1 Precision: 0.6351028216164515, 1 Recall: 0.7022739291380222
Exact: 1119, partial: 349, missing: 419, spurius: 592
PREDICTED:


In [34]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/version_2/